# Preprocess for the Taylor Diagram plots

After all of my bumbling around trying not to preprocess output, I've ended up here. A good lesson.

__I need to preprocess 3 outputs:__  
1. MISR total cloud  
2. MISR low-topped thick cloud  
3. MODIS high-topped thick cloud

### Function and package imports

In [1]:
import sys
# Add common resources folder to path
sys.path.append('/glade/u/home/jonahshaw/Scripts/git_repos/CESM2_analysis')
sys.path.append('/glade/u/home/jonahshaw/Scripts/git_repos/CESM2_analysis/Common/')
# sys.path.append("/home/jonahks/git_repos/netcdf_analysis/Common/")

from imports import (
    pd, np, xr, mpl, plt, sns, os, 
    datetime, sys, crt, gridspec,
    ccrs, metrics, Iterable, cmaps,
    mpl,glob
    )

from functions import (
    masked_average, add_weights, sp_map,
    season_mean, get_dpm, leap_year, share_ylims,
    to_png
    )

from cloud_metric import Cloud_Metric
from collections import deque
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


#### Taylor plot specific imports

In [2]:
import taylor_jshaw as taylor
import matplotlib as matplotlib
import matplotlib.patches as patches

In [3]:
from interp_functions import *

### Label appropriate directories

In [4]:
# original observations from the Kay 2012 paper
og_dir = '/glade/u/home/jonahshaw/w/kay2012_OGfiles'

In [5]:
# where to save processed files
save_dir = '/glade/u/home/jonahshaw/w/archive/taylor_files/'

In [6]:
# CAM4 and CAM5 model runs
oldcase_dir = '/glade/u/home/jonahshaw/w/archive/Kay_COSP_2012/'
# CAM6 model runs
newcase_dir = '/glade/p/cesm/pcwg/jenkay/COSP/cesm21/'

In [7]:
case_dirs = [oldcase_dir,oldcase_dir,newcase_dir]
# cases = [
#     '%s%s' % (oldcase_dir,'cam4_1deg_release_amip'),
#     '%s%s' % (oldcase_dir,'cam5_1deg_release_amip'),
#     '%s%s' % (newcase_dir,'f.e21.FHIST_BGC.f09_f09_mg17.CMIP6-AMIP.001_cosp1')    
# ]
cases = [
    'cam4_1deg_release_amip',
    'cam5_1deg_release_amip',
    'f.e21.FHIST_BGC.f09_f09_mg17.CMIP6-AMIP.001_cosp1'   
]

cosp_v = [2,2,1] # cosp version (guess)

In [8]:
# Time ranges to select by:
time_range1 = ['2001-01-01', '2010-12-31']
time_range2 = ['0001-01-01', '0010-12-31']

In [9]:
def get_file(var_name,case,suffix=''):
    return glob.glob('%s/%s/*%s*.nc' % (case,suffix,var_name))

In [10]:
def fix_cam_time(ds):
    try:
        ds['time'] = ds['time_bnds'].isel(bnds=0)
    except:
        ds['time'] = ds['time_bnds'].isel(nbnd=0)
        
    return ds

In [11]:
def select_AMIP(ds):
    
    if ds['time.year'][0] > 1000: # bad way to discriminate by year format
        _ds = ds.sel(time=slice('2001-01-01', '2010-12-31')) # work for the AMIP 0000-0010
    else:
        _ds = ds.sel(time=slice('0001-01-01', '0010-12-31')) # work for the AMIP 0000-0010
    
    return _ds

### Preprocess CLDTOT_MISR

In [12]:
suffix = 'atm/proc/tseries/month_1'
_vars = ['CLD_MISR']
_nvars = ['CLDTOT_MISR'] # new variable name
# _vars = ['CLDTOT_ISCCP','CLDTOT_MISR','CLDTOT_CAL']

model_das = {}
for j,jj in zip(_vars,_nvars):
    print(j)
    var_files = []
    for i,ii in zip(case_dirs,cases):
        print(ii)
        save_file = '%s/%s/%s.%s.nc' % (save_dir,ii,ii,jj)
#         if not os.path.exists(save_file):
        print(save_file)
#         break
        _f = glob.glob('%s/%s/%s/*%s*.nc' % (i,ii,suffix,j)) # get the correct file
        # open dataset
        _ds = xr.open_dataset(_f[0])
        # apply time bounds  
        _ds = fix_cam_time(_ds)
        # select the AMIP period
        _ds = select_AMIP(_ds)
        # Calculate the total cloud:
        _clt_misr = _ds[j].where(_ds[j].cosp_tau>0.3).sum(dim=['cosp_tau','cosp_htmisr'])
        # Save:
        _clt_misr.to_netcdf(save_file)

CLD_MISR
cam4_1deg_release_amip
/glade/u/home/jonahshaw/w/archive/taylor_files//cam4_1deg_release_amip.CLDTOT_MISR.nc


KeyboardInterrupt: 

### Preprocess CLDTHCK_MISR

In [36]:
suffix = 'atm/proc/tseries/month_1'
_vars = ['CLD_MISR']
_nvars = ['CLDTHCK_MISR'] # new variable name
# _vars = ['CLDTOT_ISCCP','CLDTOT_MISR','CLDTOT_CAL']

model_das = {}
for j,jj in zip(_vars,_nvars):
    print(j)
    var_files = []
    for i,ii,v in zip(case_dirs,cases,cosp_v):
        print(ii)
        save_file = '%s/%s/%s.%s.nc' % (save_dir,ii,ii,jj)
#         if not os.path.exists(save_file):
        print(save_file)
#         break
        _f = glob.glob('%s/%s/%s/*%s*.nc' % (i,ii,suffix,j)) # get the correct file
        # open dataset
        _ds = xr.open_dataset(_f[0])
#         break
        # apply time bounds  
        _ds = fix_cam_time(_ds)
        # select the AMIP period
        _ds = select_AMIP(_ds)
        # Calculate the total cloud:
        cells = _ds[j].where(_ds[j].cosp_tau>23)
        if v ==1:
            cells = np.bitwise_and((_ds[j].cosp_tau > 23),(_ds[j].cosp_htmisr < 3))
        else:
            cells = np.bitwise_and((_ds[j].cosp_tau > 23),(_ds[j].cosp_htmisr < 3000))
            
#         _clthck_misr = _clthck_misr.rename({'CLD_MISR':'CLDTHCK_MISR'})
        _ds[j].name = 'CLDTHCK_MISR'        
        _clthck_misr = _ds[j].where(cells).sum(dim=['cosp_tau','cosp_htmisr'])
        break
        # Save:
        _clthck_misr.to_netcdf(save_file)

CLD_MISR
cam4_1deg_release_amip
/glade/u/home/jonahshaw/w/archive/taylor_files//cam4_1deg_release_amip/cam4_1deg_release_amip.CLDTHCK_MISR.nc


In [41]:
file1 = '/glade/u/home/jonahshaw/w/archive/Kay_COSP_2012/cam4_1deg_release_amip/atm/proc/tseries/month_1/'+'cam4_1deg_release_amip.cam.h0.CLD_MISR.200011-201012.nc'
ds1 = xr.open_dataset(file1)

In [49]:
cells1 = np.bitwise_and(ds1['CLD_MISR'].cosp_tau > 23,ds1['CLD_MISR'].cosp_htmisr < 3)

clthck1 = ds1['CLD_MISR'].where(cells1).sum(['cosp_tau','cosp_htmisr'])

In [60]:
clthck1.isel(time=slice(2,None)).to_netcdf('/glade/u/home/jonahshaw/w/archive/taylor_files/cam4_1deg_release_amip/cam4_1deg_release_amip.CLDTOT_MISR.nc2')

In [43]:
file2 = '/glade/u/home/jonahshaw/w/archive/Kay_COSP_2012/cam5_1deg_release_amip/atm/proc/tseries/month_1/'+'cam5_1deg_release_amip.cam.h0.CLD_MISR.200101-201012.nc'
ds2 = xr.open_dataset(file2)

In [61]:
cells2 = np.bitwise_and(ds2['CLD_MISR'].cosp_tau > 23,ds2['CLD_MISR'].cosp_htmisr < 3)

clthck2 = ds2['CLD_MISR'].where(cells2).sum(['cosp_tau','cosp_htmisr'])

In [62]:
clthck2.to_netcdf('/glade/u/home/jonahshaw/w/archive/taylor_files/cam5_1deg_release_amip/cam5_1deg_release_amip.CLDTOT_MISR.nc2')

In [44]:
file3 = '/glade/p/cesm/pcwg/jenkay/COSP/cesm21/f.e21.FHIST_BGC.f09_f09_mg17.CMIP6-AMIP.001_cosp1/atm/proc/tseries/month_1/' + 'f.e21.FHIST_BGC.f09_f09_mg17.CMIP6-AMIP.001_cosp1.cam.h0.CLD_MISR.197901-201412.nc'
ds3 = xr.open_dataset(file3)

In [72]:
test3 = ds3.isel(time=slice(-180,-60))

In [73]:
cells3 = np.bitwise_and(test3['CLD_MISR'].cosp_tau > 23,test3['CLD_MISR'].cosp_htmisr < 3000) # this one in km?

clthck3 = test3['CLD_MISR'].where(cells3).sum(['cosp_tau','cosp_htmisr'])

In [74]:
clthck3.to_netcdf('/glade/u/home/jonahshaw/w/archive/taylor_files/f.e21.FHIST_BGC.f09_f09_mg17.CMIP6-AMIP.001_cosp1/f.e21.FHIST_BGC.f09_f09_mg17.CMIP6-AMIP.001_cosp1.CLDTOT_MISR.nc2')


In [37]:
start_dir = '/glade/u/home/jonahshaw/w/archive/taylor_files'

for i in os.listdir(start_dir):
    _path = '%s/%s' % (start_dir,i)
#     print(_path)
    _file = glob.glob(_path+'/*CLDTHCK*')[0]
    print(_file)
# #     print(os.listdir('%s/%s' % (start_dir,i)))
    _temp = xr.open_dataset(_file)
    print(_temp.mean())
#     _temp = _temp.rename({'CLD_MISR':'CLDTOT_MISR'})
#     _temp.to_netcdf('%s/%s' % (_path,_file)+'1')

/glade/u/home/jonahshaw/w/archive/taylor_files/f.e21.FHIST_BGC.f09_f09_mg17.CMIP6-AMIP.001_cosp1/f.e21.FHIST_BGC.f09_f09_mg17.CMIP6-AMIP.001_cosp1.CLDTHCK_MISR.nc
<xarray.Dataset>
Dimensions:       ()
Data variables:
    CLDTHCK_MISR  float32 35966.93
/glade/u/home/jonahshaw/w/archive/taylor_files/cam5_1deg_release_amip/cam5_1deg_release_amip.CLDTHCK_MISR.nc
<xarray.Dataset>
Dimensions:       ()
Data variables:
    CLDTHCK_MISR  float32 17983.465
/glade/u/home/jonahshaw/w/archive/taylor_files/cam4_1deg_release_amip/cam4_1deg_release_amip.CLDTHCK_MISR.nc
<xarray.Dataset>
Dimensions:       ()
Data variables:
    CLDTHCK_MISR  float32 561.9833


### Preprocess CLDTHCK_MODIS

In [82]:
suffix = 'atm/proc/tseries/month_1'
_vars = ['CLMODIS']
_nvars = ['CLDTHCK_MODIS'] # new variable name
# _vars = ['CLDTOT_ISCCP','CLDTOT_MISR','CLDTOT_CAL']

model_das = {}
for j,jj in zip(_vars,_nvars):
    print(j)
    var_files = []
    for i,ii,v in zip(case_dirs,cases,cosp_v):
        print(ii)
        save_file = '%s/%s/%s.%s.nc' % (save_dir,ii,ii,jj)
#         if not os.path.exists(save_file):
        print(save_file)
#         break
        _f = glob.glob('%s/%s/%s/*%s*.nc' % (i,ii,suffix,j)) # get the correct file
        # open dataset
        _ds = xr.open_dataset(_f[0])
        # apply time bounds  
        _ds = fix_cam_time(_ds)
        # select the AMIP period
        _ds = select_AMIP(_ds)
        # Calculate the total cloud:
        if v ==1:
            cells = np.bitwise_and((_ds[j].cosp_tau_modis > 23),(_ds[j].cosp_prs < 440))
        else:
            cells = np.bitwise_and((_ds[j].cosp_tau_modis > 23),(_ds[j].cosp_prs < 44000))
            
        _clthck_modis = _ds[j].where(cells).sum(dim=['cosp_tau_modis','cosp_prs'])
        print(_clthck_modis.mean())
        # Save:
#         _clthck_modis = _clthck_modis.rename({'CLMODIS':'CLDTHCK_MODIS'})
        _clthck_modis.to_netcdf(save_file)

CLMODIS
cam4_1deg_release_amip
/glade/u/home/jonahshaw/w/archive/taylor_files//cam4_1deg_release_amip/cam4_1deg_release_amip.CLDTHCK_MODIS.nc
<xarray.DataArray 'CLMODIS' ()>
array(17.09967, dtype=float32)
cam5_1deg_release_amip
/glade/u/home/jonahshaw/w/archive/taylor_files//cam5_1deg_release_amip/cam5_1deg_release_amip.CLDTHCK_MODIS.nc
<xarray.DataArray 'CLMODIS' ()>
array(10.560765, dtype=float32)
f.e21.FHIST_BGC.f09_f09_mg17.CMIP6-AMIP.001_cosp1
/glade/u/home/jonahshaw/w/archive/taylor_files//f.e21.FHIST_BGC.f09_f09_mg17.CMIP6-AMIP.001_cosp1/f.e21.FHIST_BGC.f09_f09_mg17.CMIP6-AMIP.001_cosp1.CLDTHCK_MODIS.nc
<xarray.DataArray 'CLMODIS' ()>
array(0., dtype=float32)


In [83]:
file1 = '/glade/u/home/jonahshaw/w/archive/Kay_COSP_2012/cam4_1deg_release_amip/atm/proc/tseries/month_1/'+'cam4_1deg_release_amip.cam.h0.CLMODIS.200011-201012.nc'
ds1 = xr.open_dataset(file1)

In [85]:
ds1 = fix_cam_time(ds1)
ds1 = select_AMIP(ds1) #['CLMODIS']

In [90]:
cells1 = np.bitwise_and(ds1['CLMODIS'].cosp_tau_modis > 23,ds1['CLMODIS'].cosp_prs < 400)

clthck1 = ds1['CLMODIS'].where(cells1).sum(['cosp_tau_modis','cosp_prs'])
print(clthck1.mean())

<xarray.DataArray 'CLMODIS' ()>
array(8.213175, dtype=float32)


In [91]:
clthck1.to_netcdf('/glade/u/home/jonahshaw/w/archive/taylor_files/cam4_1deg_release_amip/cam4_1deg_release_amip.CLDTHCK_MISR.nc2')

In [92]:
file2 = '/glade/u/home/jonahshaw/w/archive/Kay_COSP_2012/cam5_1deg_release_amip/atm/proc/tseries/month_1/'+'cam5_1deg_release_amip.cam.h0.CLMODIS.200101-201012.nc'
ds2 = xr.open_dataset(file2)

In [93]:
cells2 = np.bitwise_and(ds2['CLMODIS'].cosp_tau_modis > 23,ds2['CLMODIS'].cosp_prs < 400)

clthck2 = ds2['CLMODIS'].where(cells2).sum(['cosp_tau_modis','cosp_prs'])
print(clthck2.mean())

<xarray.DataArray 'CLMODIS' ()>
array(4.429209, dtype=float32)


In [94]:
clthck2.to_netcdf('/glade/u/home/jonahshaw/w/archive/taylor_files/cam5_1deg_release_amip/cam5_1deg_release_amip.CLDTHCK_MODIS.nc2')

In [95]:
file3 = '/glade/p/cesm/pcwg/jenkay/COSP/cesm21/f.e21.FHIST_BGC.f09_f09_mg17.CMIP6-AMIP.001_cosp1/atm/proc/tseries/month_1/' + 'f.e21.FHIST_BGC.f09_f09_mg17.CMIP6-AMIP.001_cosp1.cam.h0.CLMODIS.197901-201412.nc'
ds3 = xr.open_dataset(file3)

In [96]:
ds3 = fix_cam_time(ds3)
ds3 = select_AMIP(ds3) #['CLMODIS']

In [100]:
cells3 = np.bitwise_and(ds3['CLMODIS'].cosp_tau_modis > 23,ds3['CLMODIS'].cosp_prs < 40000)

clthck3 = ds3['CLMODIS'].where(cells3).sum(['cosp_tau_modis','cosp_prs'])
print(clthck3.mean())

<xarray.DataArray 'CLMODIS' ()>
array(2.3065884, dtype=float32)


In [101]:
clthck3.to_netcdf('/glade/u/home/jonahshaw/w/archive/taylor_files/f.e21.FHIST_BGC.f09_f09_mg17.CMIP6-AMIP.001_cosp1/f.e21.FHIST_BGC.f09_f09_mg17.CMIP6-AMIP.001_cosp1.CLDTHCK_MODIS.nc2')
